In [1]:
%pip install mediapipe

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.20.0 requires protobuf>=5.28.0, but you have protobuf 4.25.8 which is incompatible.
tensorflow-intel 2.17.0 requires ml-dtypes<0.5.0,>=0.3.1, but you have ml-dtypes 0.5.3 which is incompatible.
tensorflow-intel 2.17.0 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.20.0 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
# import tensorflow as tf
import cv2
import mediapipe
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import brown
from nltk import FreqDist
import nltk
import keras
import keras.layers as layers
import einops
import mediapipe as mp

ImportError: cannot import name 'runtime_version' from 'google.protobuf' (c:\Users\aarav\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\__init__.py)

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

In [ ]:
print("GPUs available:", tf.config.list_physical_devices('GPU'))

In [ ]:
trainDataset = keras.utils.image_dataset_from_directory(
  path+"/asl_alphabet_train/asl_alphabet_train",
  labels="inferred",
  label_mode="int",
  image_size=(224, 224),
  batch_size=1,
  shuffle=True,
  validation_split=0.2,
  subset="training",
  seed=123
)

valDataset = keras.utils.image_dataset_from_directory(
  path+"/asl_alphabet_train/asl_alphabet_train",
  labels="inferred",
  label_mode="int",
  image_size=(224, 224),
  batch_size=1,
  shuffle=True,
  validation_split=0.2,
  subset="validation",
  seed=123
)

print(trainDataset.class_names)
mp_hands = mp.solutions.hands

def cropHand(image):
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.001) as hands:
        results = hands.process(image)
        h, w, _ = image.shape
        if results.multi_hand_landmarks:
            x_min, y_min = w, h
            x_max, y_max = 0, 0
            for hand_landmarks in results.multi_hand_landmarks:
                for lm in hand_landmarks.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    x_min, y_min = min(x_min, x), min(y_min, y)
                    x_max, y_max = max(x_max, x), max(y_max, y)
            pad = 80
            x_min, y_min = max(x_min-pad, 0), max(y_min-pad, 0)
            x_max, y_max = min(x_max+pad, w), min(y_max+pad, h)
            cropped = image[y_min:y_max, x_min:x_max]

            cropped_resized = cv2.resize(cropped, (224,224))
            return cropped_resized
        else:
            image *= 0
            return cv2.resize(image, (224,224))

def preprocessWithMediapipe(x, y):
    img = x.numpy().astype(np.uint8)

    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    # img = img.squeeze(0)
    imgCropped = cropHand(img)  # cropHand expects RGB input

    imgGray = cv2.cvtColor(imgCropped, cv2.COLOR_RGB2GRAY)
    imgGray = imgGray / 255.0
    imgGray = np.expand_dims(imgGray, axis=-1).astype(np.float32)

    return imgGray, y


def tfPreprocess(x, y):
    img, label = tf.py_function(preprocessWithMediapipe, [x[0], y[0]], [tf.float32, tf.int32])
    img.set_shape([224, 224, 1])
    label.set_shape([])
    print(img.shape)
    return img, label

AUTOTUNE = tf.data.AUTOTUNE

trainDataset = trainDataset.map(tfPreprocess, num_parallel_calls=AUTOTUNE).batch(32).prefetch(AUTOTUNE)
valDataset = valDataset.map(tfPreprocess, num_parallel_calls=AUTOTUNE).batch(32).prefetch(AUTOTUNE)


In [ ]:
for img, label in trainDataset.take(1):
    print(img.shape)  # should be (32,224,224,1)

In [ ]:
def createModel():
  model = keras.models.Sequential([
    layers.Input(shape=(224, 224, 1)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(29, activation='softmax')
  ])

  return model

# model = createModel()
model = keras.saving.load_model("/content/model.keras")

In [ ]:
class BatchCheckpoint(keras.callbacks.Callback):
    def __init__(self, save_path, save_every_n_batches=100):
        super().__init__()
        self.save_path = save_path
        self.save_every_n_batches = save_every_n_batches
        self.batch_count = 0

    def on_batch_end(self, batch, logs=None):
        self.batch_count += 1
        if self.batch_count % self.save_every_n_batches == 0:
            self.model.save(f"{self.save_path}_batch_{self.batch_count}.keras")
            print(f"Saved checkpoint at batch {self.batch_count}")

checkpoint_callback = BatchCheckpoint(save_path="model_checkpoint", save_every_n_batches=100)

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    x = trainDataset,
    epochs = 100,
    validation_data = valDataset,
    callbacks=[checkpoint_callback]
)

In [ ]:
model.save("model.keras")